In [4]:
import pandas as pd
import random
import numpy as np
import scanpy as sc
# import scipy
from tqdm import tqdm

data_path = '/data2/hratch/immune_CCI/covid/covid_atlas/'
load_h5 = True

In [ ]:
pbmc_covid = sc.read_mtx(data_path + 'raw/GSE158055_covid19_counts.mtx.gz') # raw counts
if load_h5: 
    pch5 = sc.read_h5ad(data_path + 'raw/COVID19_ALL.h5ad') # load dataset
# pbmc_covid = sc.read_10x_mtx(data_path + 'raw/counts/')

print('Finished loading covid datasets')

In [39]:
# exclude samples with fewer than 2000 cells
md_cell = pd.read_csv(data_path + 'raw/GSE158055_cell_annotation.csv.gz')
n_samples = md_cell.sampleID.value_counts()
samples_to_keep = n_samples[n_samples > 2000].index.tolist()




In [40]:
md = pd.read_excel(data_path + 'raw/GSE158055_sample_metadata.xlsx', sheet_name = 0, skiprows=20)
md = md.iloc[range(304 - 20), range(25)]
md = md[md['Sample name'].isin(samples_to_keep)]


contexts = md['characteristics: CoVID-19 severity'].unique()
n_contexts = contexts.shape[0]

context_counts = md['characteristics: CoVID-19 severity'].value_counts() 
min_context_type = context_counts[context_counts == context_counts.min()].index.tolist()[0]
min_context_count = len(md[md['characteristics: CoVID-19 severity'] == min_context_type]['Patients'].unique())

max_samples = min_context_count*n_contexts

context_map = {context: md[md['characteristics: CoVID-19 severity'] == \
                           context][['Sample name', 'Patients']].reset_index(drop = True) for context in contexts}

In [41]:
# randomly select samples subsetted from the entire dataset
# make sure to choose an even number of each context
# make sure not to repeat patients within a context

n_iter = 1 # number of times to run subsetting
seed = 0

Samples = pd.DataFrame(columns = ['iteration', 'n_samples', 'sample_names'])
idx = 0

sample_iters = [3, 6, 12, 24, 36, 48, 60, 75]#list(range(n_contexts, max_samples + 1, n_contexts))

for iteration in range(n_iter):
    for n_samples in sample_iters:
        cmap_temp = context_map.copy()
        n_sample_per_context = int(n_samples/n_contexts)
        samples = list()
        for context in contexts:
            df = context_map[context].sample(frac=1, random_state = seed).drop_duplicates(subset = 'Patients') # shuffle rows to randomly drop duplicates
            samples += df.sample(n = n_sample_per_context, random_state = seed)['Sample name'].tolist()
            seed += 1
        Samples.loc[idx,:] = [iteration, n_samples, samples]
        idx += 1

Samples['sample_names'] = Samples.sample_names.apply(lambda x: '; '.join(x))            
Samples.to_csv(data_path + 'interim/timing_inputs/samples_for_timing.csv')            

/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/pandas/core/internals/blocks.py:993: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.array(value)


In [42]:
#cell_ids = pd.read_csv(data_path + 'raw/GSE158055_covid19_barcodes.tsv.gz', header = None)
gene_ids = pd.read_csv(data_path + 'raw/GSE158055_covid19_features.tsv.gz', header = None)
md_cell.set_index('cellName', drop = True, inplace = True)

pbmc_covid = pbmc_covid.transpose() 

In [ ]:
if load_h5:
    pbmc_covid.obs = pch5.obs
    pbmc_covid.var = pch5.var
else:
    pbmc_covid.obs = md_cell
    pbmc_covid.var = gene_ids.set_index(0, drop = True)

In [46]:
# split by sample id 

def flatten_list(t):
    return [item for sublist in t for item in sublist]

def create_raw_counts(sample_id):
    df = pbmc_covid[pbmc_covid.obs.sampleID == sample_id]
    sc.pp.filter_cells(df, min_genes=50) 
#     sc.pp.filter_genes(df, min_cells = 3) # avoid filtering genes, will need intersection of remaining genes, which filters to many out when subsequently filtering for LR pairs
    return df

sample_ids = list(set(flatten_list([sn.split('; ') for sn in Samples.sample_names.tolist()])))
sample_counts = {sample_id: create_raw_counts(sample_id) for sample_id in tqdm(sample_ids)}


min_cells_to_keep = min([df.n_obs for sample_id, df in sample_counts.items()])


  0%|          | 0/140 [00:00<?, ?it/s]Trying to set attribute `.obs` of view, copying.

  1%|          | 1/140 [00:06<14:38,  6.32s/it]Trying to set attribute `.obs` of view, copying.

  1%|▏         | 2/140 [00:06<06:49,  2.97s/it]Trying to set attribute `.obs` of view, copying.

  2%|▏         | 3/140 [00:07<04:15,  1.86s/it]Trying to set attribute `.obs` of view, copying.

  3%|▎         | 4/140 [00:08<03:10,  1.40s/it]Trying to set attribute `.obs` of view, copying.

  4%|▎         | 5/140 [00:08<02:09,  1.04it/s]Trying to set attribute `.obs` of view, copying.

  4%|▍         | 6/140 [00:08<01:48,  1.24it/s]Trying to set attribute `.obs` of view, copying.

  5%|▌         | 7/140 [00:09<01:41,  1.31it/s]Trying to set attribute `.obs` of view, copying.

  6%|▌         | 8/140 [00:10<01:32,  1.43it/s]Trying to set attribute `.obs` of view, copying.

  6%|▋         | 9/140 [00:10<01:28,  1.48it/s]Trying to set attribute `.obs` of view, copying.

  7%|▋         | 10/140 [00:11<01:21,

 58%|█████▊    | 81/140 [01:41<05:06,  5.20s/it]Trying to set attribute `.obs` of view, copying.

 59%|█████▊    | 82/140 [01:43<04:20,  4.49s/it]Trying to set attribute `.obs` of view, copying.

 59%|█████▉    | 83/140 [01:46<03:38,  3.84s/it]Trying to set attribute `.obs` of view, copying.

 60%|██████    | 84/140 [01:46<02:35,  2.78s/it]Trying to set attribute `.obs` of view, copying.

 61%|██████    | 85/140 [01:46<01:53,  2.07s/it]Trying to set attribute `.obs` of view, copying.

 61%|██████▏   | 86/140 [01:47<01:22,  1.52s/it]Trying to set attribute `.obs` of view, copying.

 62%|██████▏   | 87/140 [02:03<05:11,  5.87s/it]Trying to set attribute `.obs` of view, copying.

 63%|██████▎   | 88/140 [02:07<04:35,  5.29s/it]Trying to set attribute `.obs` of view, copying.

 64%|██████▎   | 89/140 [02:11<04:08,  4.87s/it]Trying to set attribute `.obs` of view, copying.

 64%|██████▍   | 90/140 [02:18<04:36,  5.52s/it]Trying to set attribute `.obs` of view, copying.

 65%|██████▌   | 91/

In [47]:
# seed = 24
seed += 1

# subset to min cells_to_keep and write to csv
cells_to_keep = list()
for sample_id in tqdm(sample_counts):
    random.seed(seed)
    df = sample_counts[sample_id]
    df = df[df.obs.index.isin(random.sample(df.obs.index.tolist(), min_cells_to_keep))] # subset
#     df.to_df().to_csv(data_path + 'interim/umi_for_timing/' + sample_id + '.csv') # write
    df.to_df().to_hdf(data_path + 'interim/timing_inputs/umi_per_sample.h5', key = sample_id)
    cells_to_keep += df.obs.index.tolist()
    seed += 1

pbmc_covid.obs[pbmc_covid.obs.index.isin(cells_to_keep)].to_csv(data_path + 'interim/timing_inputs/metadata_for_timing.csv')


  0%|          | 0/140 [00:00<?, ?it/s]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S054'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

  1%|          | 1/140 [00:02<05:33,  2.40s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M038'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

  1%|▏         | 2/140 [00:04<05:24,  2.35s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python ident


 14%|█▍        | 20/140 [00:48<04:59,  2.50s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M073'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 15%|█▌        | 21/140 [00:51<04:53,  2.46s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S030'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 16%|█▌        | 22/140 [00:53<04:48,  2.44s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid P


 29%|██▊       | 40/140 [01:37<04:12,  2.52s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M034'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 29%|██▉       | 41/140 [01:40<04:10,  2.53s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M037'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 30%|███       | 42/140 [01:42<04:04,  2.50s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid P


 43%|████▎     | 60/140 [02:27<03:18,  2.49s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M074-2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 44%|████▎     | 61/140 [02:30<03:18,  2.51s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S069-3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 44%|████▍     | 62/140 [02:32<03:16,  2.52s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a val


 57%|█████▋    | 80/140 [03:17<02:24,  2.40s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M021'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 58%|█████▊    | 81/140 [03:19<02:22,  2.41s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M045'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 59%|█████▊    | 82/140 [03:23<02:32,  2.62s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid P


 71%|███████▏  | 100/140 [04:13<01:37,  2.44s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M079'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 72%|███████▏  | 101/140 [04:16<01:34,  2.43s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-HC020-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 73%|███████▎  | 102/140 [04:18<01:32,  2.43s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a v


 86%|████████▌ | 120/140 [05:03<00:48,  2.45s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S046'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 86%|████████▋ | 121/140 [05:05<00:46,  2.44s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M024'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)

 87%|████████▋ | 122/140 [05:08<00:45,  2.53s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a vali


100%|██████████| 140/140 [05:54<00:00,  2.53s/it]


# Classification

In [17]:
md = pd.read_excel(data_path + 'raw/GSE158055_sample_metadata.xlsx', sheet_name = 0, skiprows=20)
md = md.iloc[range(304 - 20), range(25)]
md = md[md['Sample name'].isin(samples_to_keep)] # still filter for datasets with > 2000 cells

In [27]:
sample_ids = md['Sample name'].unique().tolist()
pbmc_covid.obs[pbmc_covid.obs.sampleID.isin(sample_ids)].to_csv(data_path + 'interim/classification_inputs/metadata.csv')
sample_counts = {sample_id: create_raw_counts(sample_id) for sample_id in tqdm(sample_ids)}

In [37]:
for sample_id in tqdm(sample_counts):
    df = sample_counts[sample_id]
#     df.to_df().to_csv(data_path + 'interim/umi_for_classification/' + sample_id + '.csv') # write
    df.to_df().to_hdf(data_path + 'interim/classification_inputs/umi_per_sample.h5', key = sample_id)

  2%|▏         | 4/227 [00:17<18:54,  5.09s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M043-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
  2%|▏         | 5/227 [00:25<21:38,  5.85s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M048'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
  3%|▎         | 6/227 [00:32<22:51,  6.20s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pytho

 19%|█▉        | 44/227 [04:36<15:02,  4.93s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S062'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 20%|█▉        | 45/227 [04:39<13:33,  4.47s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S063'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 20%|██        | 46/227 [04:43<12:46,  4.23s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyth

 28%|██▊       | 64/227 [05:52<10:15,  3.78s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S035-3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 29%|██▊       | 65/227 [05:54<09:08,  3.38s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S035-4'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 29%|██▉       | 66/227 [05:58<08:54,  3.32s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Py

 46%|████▌     | 104/227 [11:22<12:15,  5.98s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M028'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 46%|████▋     | 105/227 [11:25<10:30,  5.17s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M029'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 47%|████▋     | 106/227 [11:29<09:51,  4.89s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid P

 55%|█████▍    | 124/227 [13:24<10:49,  6.30s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M010-6'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 55%|█████▌    | 125/227 [13:31<10:56,  6.44s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M009-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 56%|█████▌    | 126/227 [13:39<11:52,  7.06s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid

 63%|██████▎   | 144/227 [15:52<10:23,  7.51s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M007-4'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 64%|██████▍   | 145/227 [15:56<08:55,  6.53s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M007-5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 64%|██████▍   | 146/227 [16:03<08:57,  6.64s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid

 72%|███████▏  | 164/227 [17:52<07:07,  6.78s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M017'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 73%|███████▎  | 165/227 [18:01<07:39,  7.42s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M019'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 73%|███████▎  | 166/227 [18:10<07:50,  7.71s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyt

 90%|████████▉ | 204/227 [23:41<02:21,  6.15s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S078'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 90%|█████████ | 205/227 [23:47<02:19,  6.35s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S079'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 91%|█████████ | 206/227 [23:53<02:08,  6.12s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyt